In [14]:
import duckdb
import pandas as pd
import os
from datetime import datetime

In [15]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [26]:
arquivo = 'z0019_1.csv'
#arquivo = 'z0019_2.csv'

In [27]:
data_ingestao = datetime.now()
df = pd.read_csv(f'../landing/{arquivo}', sep=';')
df['nome_arquivo'] = arquivo    
df['data_ingestao'] = data_ingestao
df.head()

,NATBR,MAKTX,WEKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10003,PREGO,BT10,100,50,z0019_1.csv,2025-07-20 18:52:25.989181
1,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-07-20 18:52:25.989181
2,10002,MARTELO,BT10,100,1500,z0019_1.csv,2025-07-20 18:52:25.989181


In [28]:
con.execute("""
CREATE TABLE IF NOT EXISTS bronze_produtos (
    NATBR VARCHAR,
    MAKTX VARCHAR,
    WEKS VARCHAR,
    MAINS VARCHAR,
    LABST VARCHAR,
    nome_arquivo VARCHAR,
    data_ingestao TIMESTAMP
);
""")

In [29]:
con.execute("INSERT INTO bronze_produtos SELECT * FROM df")

In [30]:
con.execute("""ALTER TABLE bronze_produtos RENAME TO bronze_z0019""")

CatalogException: Catalog Error: Could not rename "bronze_produtos" to "bronze_z0019": another entry with this name already exists!

In [31]:
resultado = con.execute("SELECT * FROM bronze_z0019").fetchdf()
resultado.head()

,NATBR,MAKTX,WEKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10003,PREGO,BT10,100,50,z0019_1.csv,2025-07-20 18:44:09.838079
1,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-07-20 18:44:09.838079
2,10002,MARTELO,BT10,100,1500,z0019_1.csv,2025-07-20 18:44:09.838079
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-07-20 18:44:38.241407
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-07-20 18:44:38.241407


In [32]:
con.close()